# A OOP approach to hierarchically indexed data tables 
This notebook defines the schema and relations that will be implemented to facilitate the workflows projecting global data sets on a hierarchical index and manipulating different layers together. 
Mainly, this approach aims to solve the following challenges:
- User friendly : ideally have a familiar sytax to pandas users
- Scalable : be able to work with national, continetal and even global data sets of fine spatial resolution (ex: 100m,1000m grids)
- Flexible : it should provide a easily exportable type of data to use across different types of processes and analyses.

The main tools chosen to do this are : 

* Using H3 : https://uber.github.io/h3-py/
* Using DuckDB : https://duckdb.org/docs/guides/python/execute_sql 
* Using ibis : https://ibis-project.org 



## Base workflow


One of the main contributions of this workflow is a method to efficiently project raster grids of any size into a format that is more efficient to work with and perform geometric operations with other types. 

## Minimal input

[polars.DataFrame, pyarrow.Table, pandas.DataFrame, geopandas.GeoDataFrame(not recommended), ibis.table (recommended)]

WIth non ambiguous columns for the coordinates, ex ('x','y'), ('lon','lat') etc .... and variables containing numeric values with a suffix *var* or containing categorical values with suffix *cat* preceding the actual column name *{band}*.


| lon   | lat   | {band}_var  |    ...        |
|-------|-------|-------------|---------------|
| float | float | [float,int] | ...   |



| lon   | lat   | {band}_cat | ... |
|-------|-------|----------|-----|
| float | float | str      | ... |

## Minimal output

The data is projected to the h3 grid for a resolution which is refered to later on as the native resolution. It should be the highest possible resolution that makes sense for a given data layer. For example for points data, any resolution is possible, and the choise will most likely depend on the type of processing that is done with it. 
In the case of polygons, a resolution high enough to give a good description of the original shape, but coarse enough to be efficient to work with. Once again, this is context specific and will most likely depend on the type of analysis.


| lon   | lat   | {band}_cat | ... | h3_id     |
|-------|-------|----------|-----|-----------|
| float | float | str      | ... | [str,int] |

| lon   | lat   | {band}_var  |    ... |   h3_id   |
|-------|-------|-------------|--------|-----------|
| float | float | [float,int] | ...    | [str,int] |


A mix of the two previous tyopes of numeric or categorical variables can be present as long as the *coordinates* and *h3_id* columns are unambiguiously identified. 

The object to which the data is associated contains additional attributes and methods descrbed next:

## Class

### Attributes
- base_res : base resolution, the original resolution into which the data was projected on the h3 grid
- current_res : current resolution, the resolution in which the data currently is after potential processing. 
### Methods
- set_res(res) : int(4,18), set the resolution of the data to a specifed one.
- change_res(level) : int, change the resolution of the data by a value provided. the reuslting resolution will be equal to *current_res*+*level*
- add_layer()

## In practice

In [14]:
import scalenav.oop as snoo
import pandas as pd
import numpy.random as rd
import ibis as ib
import ibis.selectors as s
from ibis import _

ib.options.interactive = True

In [2]:
conn = snoo.ScalenavProcess()

Connecting to a temporary in-memory DB instance.
Connecting to a temporary in-memory DB instance.


In [3]:
size = 10
tab1 = pd.DataFrame({"a_var" : rd.uniform(0,10,size),
"lon" : rd.uniform(0,180,size),
"lat" : rd.uniform(-90,90,size),
})

# return an ibis table
tab1 = conn.create_table(obj=tab1,name="tab1",overwrite = True)

In [4]:
tab1

┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ a_var    ┃ lon        ┃ lat        ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64  │ float64    │ float64    │
├──────────┼────────────┼────────────┤
│ 8.049506 │  17.125325 │ -56.296328 │
│ 0.334255 │  57.349014 │  80.817268 │
│ 6.471203 │  86.807205 │ -57.225707 │
│ 7.528599 │  58.095845 │  82.683652 │
│ 5.197592 │  89.105492 │  33.476202 │
│ 9.912825 │ 156.084059 │ -85.837527 │
│ 6.605455 │ 147.971272 │  69.892358 │
│ 4.518133 │ 174.344491 │  60.127325 │
│ 3.248270 │  77.541509 │  65.233396 │
│ 7.714036 │  54.660095 │ -35.042960 │
└──────────┴────────────┴────────────┘

In [15]:
tab1_h3 = snoo.sn_project(tab1).select("h3_id",s.matches("_var"),)
snoo.sn_head(tab1_h3)

Assuming coordinates columns ('lon','lat')
Assuming coordinates columns ('lon','lat')


┌────┐
│ 10 │
└────┘
┌────┐
│ 10 │
└────┘


┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ h3_id           ┃ a_var    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string          │ float64  │
├─────────────────┼──────────┤
│ 88e6d64137fffff │ 8.049506 │
│ 88001a003bfffff │ 0.334255 │
│ 88e4ea6115fffff │ 6.471203 │
│ 8800f5bb41fffff │ 7.528599 │
│ 883d58d065fffff │ 5.197592 │
└─────────────────┴──────────┘
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ h3_id           ┃ a_var    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string          │ float64  │
├─────────────────┼──────────┤
│ 88e6d64137fffff │ 8.049506 │
│ 88001a003bfffff │ 0.334255 │
│ 88e4ea6115fffff │ 6.471203 │
│ 8800f5bb41fffff │ 7.528599 │
│ 883d58d065fffff │ 5.197592 │
└─────────────────┴──────────┘


In [16]:
tab1_ds = snoo.sn_change_res(tab1_h3,levels=2)
snoo.sn_head(tab1_ds)

┌─────┐
│ 490 │
└─────┘
┌─────┐
│ 490 │
└─────┘


┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ a_var    ┃ h3_id           ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ float64  │ string          │
├──────────┼─────────────────┤
│ 0.006822 │ 8a001a003a07fff │
│ 0.006822 │ 8a001a003a0ffff │
│ 0.006822 │ 8a001a003a17fff │
│ 0.006822 │ 8a001a003a1ffff │
│ 0.006822 │ 8a001a003a27fff │
└──────────┴─────────────────┘
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ a_var    ┃ h3_id           ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ float64  │ string          │
├──────────┼─────────────────┤
│ 0.006822 │ 8a001a003a07fff │
│ 0.006822 │ 8a001a003a0ffff │
│ 0.006822 │ 8a001a003a17fff │
│ 0.006822 │ 8a001a003a1ffff │
│ 0.006822 │ 8a001a003a27fff │
└──────────┴─────────────────┘


In [19]:
tab1_ = snoo.sn_change_res(tab1_ds,levels=-2)
snoo.sn_head(tab1_)

┌────┐
│ 10 │
└────┘
┌────┐
│ 10 │
└────┘


┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ h3_id           ┃ a_var    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string          │ float64  │
├─────────────────┼──────────┤
│ 8800f5bb41fffff │ 7.528599 │
│ 88cb09696dfffff │ 7.714036 │
│ 8804a91a6bfffff │ 6.605455 │
│ 880b0b85ddfffff │ 3.248270 │
│ 88f2b68dc3fffff │ 9.912825 │
└─────────────────┴──────────┘
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ h3_id           ┃ a_var    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string          │ float64  │
├─────────────────┼──────────┤
│ 8800f5bb41fffff │ 7.528599 │
│ 88cb09696dfffff │ 7.714036 │
│ 8804a91a6bfffff │ 6.605455 │
│ 880b0b85ddfffff │ 3.248270 │
│ 88f2b68dc3fffff │ 9.912825 │
└─────────────────┴──────────┘


In [23]:
ib.difference(tab1_.select("a_var"), tab1_h3.select("a_var"))

┏━━━━━━━━━━┓
┃ a_var    ┃
┡━━━━━━━━━━┩
│ float64  │
├──────────┤
│ 7.714036 │
│ 0.334255 │
│ 3.248270 │
│ 5.197592 │
│ 8.049506 │
│ 4.518133 │
│ 7.528599 │
│ 9.912825 │
│ 6.471203 │
└──────────┘

In [24]:
7**(-2)

0.02040816326530612